In [13]:
import pandas as pd
import numpy as np
import json
import ast
import re
from Utils import helper as hp
import pyarrow as pyarrow
import pyarrow.parquet as pq

In [3]:
user_parquet = pq.ParquetFile(r'data-states\user.parquet')
for batch in user_parquet.iter_batches():
    batch_df = batch.to_pandas()

In [4]:
hp.data_overview(batch_df)

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,user_id,[<class 'str'>],100.0,0.0,0
1,name,[<class 'str'>],100.0,0.0,0
2,review_count,[<class 'int'>],100.0,0.0,0
3,yelping_since,[<class 'str'>],100.0,0.0,0
4,useful,[<class 'int'>],100.0,0.0,0
5,funny,[<class 'int'>],100.0,0.0,0
6,cool,[<class 'int'>],100.0,0.0,0
7,elite,[<class 'str'>],100.0,0.0,0
8,friends,[<class 'str'>],100.0,0.0,0
9,fans,[<class 'int'>],100.0,0.0,0


In [5]:
batch_df.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [6]:
batch_df[batch_df.duplicated()]

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos


In [7]:
outliers_columns = ["useful","funny","cool",'fans',"average_stars"]

for column in outliers_columns:
    hp.outliers_percentage(batch_df, column)
    print("-"*66)

Total de outliers en useful: 91 de 8445 registros -- 1.08 %
------------------------------------------------------------------
Total de outliers en funny: 79 de 8445 registros -- 0.94 %
------------------------------------------------------------------
Total de outliers en cool: 71 de 8445 registros -- 0.84 %
------------------------------------------------------------------
Total de outliers en fans: 146 de 8445 registros -- 1.73 %
------------------------------------------------------------------
Total de outliers en average_stars: 200 de 8445 registros -- 2.37 %
------------------------------------------------------------------
